In [1]:
import numpy as np
import pandas as pd
import time
from cvxopt import matrix
from cvxopt import solvers

In [2]:
X = pd.read_csv('spambase.data',header=None).to_numpy()

In [3]:
#set random seed
np.random.seed(1023)
#turn off output of qp solver
solvers.options['show_progress'] = False

In [4]:
#change labels to -1 and 1
X[:,57] = 2*X[:,57] - 1

In [5]:
#separate training and test data
#get balanced split
split = .67
df_spam=X[X[:,57]==1,:]
df_nospam = X[X[:,57]==-1,:]
spam_size = int(np.floor(split*df_spam.shape[0]))
nospam_size = int(np.floor(split*df_nospam.shape[0]))
tr_spm_idx = np.random.choice(range(df_spam.shape[0]),size = spam_size,replace=False)
te_spm_idx = np.setdiff1d(range(df_spam.shape[0]),tr_spm_idx)
tr_nspm_idx = np.random.choice(range(df_nospam.shape[0]),size=nospam_size,replace=False)
te_nspm_idx = np.setdiff1d(range(df_nospam.shape[0]),tr_nspm_idx)
df_train = np.concatenate([df_spam[tr_spm_idx,:], df_nospam[tr_nspm_idx,:]],axis=0)
df_test = np.concatenate([df_spam[te_spm_idx,:], df_nospam[te_nspm_idx,:]],axis=0)

N = X.shape[0]
#get dimension of data
d = X.shape[1]
idx = np.random.choice(np.arange(N),size=int(np.floor(split*N)),replace=False)
test_idx = np.setdiff1d(np.arange(N), idx)
#split data
X_train = df_train[:,0:57]
X_test = df_test[:,0:57]
#split labels
y_train = df_train[:,57]
y_test = df_test[:,57]
#set up some constants
N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [6]:
#standardize data for better results
mean = np.mean(X_train,axis=0)
std = np.std(X_train,axis=0)
X_train = 1/std*(X_train-mean)
X_test = 1/std*(X_test- mean)

In [7]:
#set sigmoid kernel parameters 
sigmoid_gamma = 1/N_train
sigmoid_kappa = 0

In [8]:
#kernels
#Gaussian kernel
def K1(x,y,gamma=1):
    return np.exp(-gamma*np.linalg.norm(x-y,axis=1)**2)
#polynomial kernel
def K2(x,y,gamma=1,d=2):
    return (gamma*np.sum(x*y,axis=1))**d
#linear kernel
def K3(x,y,gamma=1):
    return gamma*np.sum(x*y,axis=1)
#sigmoid kernel
def K4(x,y,gamma=sigmoid_gamma,kappa=sigmoid_kappa):
    return np.tanh(gamma*np.sum(x*y,axis=1)+kappa)

In [9]:
#functions for gram matrices 
#Gaussian kernel GM
def gramK1(X,gamma=1):
    return np.exp(-gamma*np.linalg.norm(X.T-X[:,:,None],axis=1)**2)
#polynomial kernel gram matrix
def gramK2(X,gamma=0,d=2):
    return (np.sum(X.T*X[:,:,None],axis=1)*gamma)**d
#linear kernel GM
def gramK3(X,gamma=0):
    return np.sum(X.T*X[:,:,None],axis=1)*gamma
#sigmoid GM
def gramK4(X,gamma=sigmoid_gamma,kappa=sigmoid_kappa):
    return np.tanh(gamma*np.sum(X.T*X[:,:,None],axis=1)+ kappa)

In [11]:
def optimize(gram,y_train=y_train,N_train=N_train):
    #set up and solve SVM optimization problem
    P = matrix(y_train[:,None]@y_train[None,:]*gram)
    q = matrix(-np.ones(N_train))
    G = matrix(np.concatenate([np.eye(N_train),-np.eye(N_train)],axis=0))
    h = matrix(np.concatenate([C*np.ones(N_train),np.zeros(N_train)]))
    A = matrix(y_train[None,:])
    b = matrix(np.zeros(1)[None,:])
    sol = solvers.qp(P,q,G,h,A,b)
    #get solution and convert to numpy
    return np.array(sol['x'])    

In [10]:
def predict(x,alpha,K,b,gamma,N_train=N_train,X=X_train,y=y_train):
    #x_rep = np.repeat(x,N_train,axis=0)
    kern_vec = K(x,X,gamma)
    return np.sign(alpha.T@(y*kern_vec)+b)

In [13]:
def evaluate(alpha,K,gram,gamma,X_train=X_train,X_test=X_test,N_test=N_test,y_train=y_train,N_train=N_train):
    #compute offset b
    #idx = np.where(alpha!=0)[0]
    #get only vectors with large alpha's (support vectors)
    epsilon = np.quantile(alpha,.9)
    idx = np.where(alpha > epsilon)[0]
    gram_idx = gram[:,idx]
    yi = y_train[idx]
    N_s = len(idx)
    b = np.mean(yi[None,:] - (alpha*y_train[:,None]).T@gram_idx)
    #evalutate model
    num_correct = 0
    for i in range(N_test):
        pred = predict(X_test[None,i,:],alpha,K,b,gamma)
        if abs(y_test[i] - pred)<.1:
            num_correct = num_correct + 1
    return 100*num_correct/N_test

In [14]:
#kernel dictionary
kern_dict = {'gaussian':'K1','polynomial':'K2','linear':'K3','sigmoid':'K4'}
#Gram matrix dictionary
GM_dict = {'gaussian':'gramK1','polynomial':'gramK2', 'linear':'gramK3','sigmoid':'gramK4'}

In [15]:
#stolen from sam
Cs_to_test = [.001,.01,.1,1.,10.,100.0,1000.0,10.**4,10.**5]
gammas_to_test = [.01,.1,.5,1.,10.,100.]
results = pd.DataFrame(columns=['kernel','C','gamma','accuracy','comp time'])
for kernel in kern_dict.keys():
    #set kernel 
    K = eval(kern_dict[kernel])
    #specify gram matrix function to use
    GM = eval(GM_dict[kernel])
    #regularization hyperparameter
    for C in Cs_to_test:
        #don't test a range of gammas for sigmoid kernel.  Sigmoid kernels don't work for every gamma setting
        if kernel != 'sigmoid':
            for gamma in gammas_to_test:
                start_time = time.time()
                #get Gram matrix
                #Technically, we don't have to compute the Gram matrix for each C; just once is enough
                #But we want computation time to include forming the Gram matrix
                gram = GM(X_train,gamma)
                print('kernel:' + kernel + " \tC:" + str(C) + " \tgamma:" + str(gamma))
                #optimize SVM
                alpha = optimize(gram)
                #get run time
                run_time = time.time() - start_time
                #evalutate model
                acc = evaluate(alpha,K,gram,gamma)
                #save results
                results = results.append({'kernel':kernel, 'C':C,'gamma':gamma,'accuracy':acc,'comp time':run_time},ignore_index=True)
        else:
            start_time = time.time()
            #set gamma to default sigmoid gamma parameter
            gamma = sigmoid_gamma
            gram = GM(X_train,gamma)
            print('kernel:' + kernel + " \tC:" + str(C) + " \tgamma:" + str(gamma))
            alpha = optimize(gram)
            run_time = time.time() - start_time
            acc = evaluate(alpha,K,gram,gamma)
            results = results.append({'kernel':kernel, 'C':C,'gamma':gamma,'accuracy':acc,'comp time':run_time},ignore_index=True)

kernel:gaussian 	C:0.001 	gamma:0.01
kernel:gaussian 	C:0.001 	gamma:0.1
kernel:gaussian 	C:0.001 	gamma:0.5
kernel:gaussian 	C:0.001 	gamma:1.0
kernel:gaussian 	C:0.001 	gamma:10.0
kernel:gaussian 	C:0.001 	gamma:100.0
kernel:gaussian 	C:0.01 	gamma:0.01
kernel:gaussian 	C:0.01 	gamma:0.1
kernel:gaussian 	C:0.01 	gamma:0.5
kernel:gaussian 	C:0.01 	gamma:1.0
kernel:gaussian 	C:0.01 	gamma:10.0
kernel:gaussian 	C:0.01 	gamma:100.0
kernel:gaussian 	C:0.1 	gamma:0.01
kernel:gaussian 	C:0.1 	gamma:0.1
kernel:gaussian 	C:0.1 	gamma:0.5
kernel:gaussian 	C:0.1 	gamma:1.0
kernel:gaussian 	C:0.1 	gamma:10.0
kernel:gaussian 	C:0.1 	gamma:100.0
kernel:gaussian 	C:1.0 	gamma:0.01
kernel:gaussian 	C:1.0 	gamma:0.1
kernel:gaussian 	C:1.0 	gamma:0.5
kernel:gaussian 	C:1.0 	gamma:1.0
kernel:gaussian 	C:1.0 	gamma:10.0
kernel:gaussian 	C:1.0 	gamma:100.0
kernel:gaussian 	C:10.0 	gamma:0.01
kernel:gaussian 	C:10.0 	gamma:0.1
kernel:gaussian 	C:10.0 	gamma:0.5
kernel:gaussian 	C:10.0 	gamma:1.0
kernel:ga

In [16]:
results.to_csv('results_balanced.csv',index=False)